# Preprocessing

## import, 데이터 불러오기

In [1]:
import numpy as np
import pandas as pd
import json

In [2]:
df_trend = pd.read_csv('data/검색량/총검색량.csv')

In [3]:
df_subway = pd.read_csv('data/지하철/subway_raw.csv')
df_subway = df_subway[['Date', 'place', 'total']]
df_subway['place'].replace('종로', '익선', inplace=True) # 장소명 통일을 위해 종로를 익선으로 변경

# 데이터 병합
네이버 트렌드(검색량) 데이터, 지하철 이용량 데이터, 부동산 데이터를 하나의 데이터프레임으로 병합.  
부동산 매매가의 경우 월별 데이터밖에 없으므로 아래 두개의 데이터프레임을 생성할 것임.
- 일별 : 검색량 - 지하철 이용량 병합
- 월별 : 검색량 - 지하철 이용량 - 부동산 매매가 병합


## 일별
지하철 이용량 데이터가 2021년 8월 31일까지만 있어서 이 기간까지의 데이터만 사용  
- 데이터 : 네이버 트렌드 검색량, 지하철 이용량
- 기간 : 2016-01-01 ~ 2021-08-31

### Reshape
트렌드(검색량) 데이터와 지하철 이용량 데이터를 합치기 위해 데이터 프레임 형태를 맞춰줌

In [4]:
# stack을 사용하여 데이터프레임 reshape
df_t = df_trend.set_index('period').stack().reset_index()

# 컬럼명 통일
df_t.rename(columns={'period':'Date','level_1':'place', 0:'trend'}, inplace=True)
df_t.head()

,Date,place,trend
0,2016-01-01,홍대,47514.094171
1,2016-01-01,연남,24403.184823
2,2016-01-01,이태원,26406.034126
3,2016-01-01,익선,23092.369742
4,2016-01-01,을지로,2298.162711


In [5]:
# 데이터 프레임 
df_temp = df_subway.groupby(['Date', 'place']).sum()
df_temp = pd.pivot_table(df_temp, values = 'total', index = 'Date', columns='place')

# stack을 사용하여 reshape
df_s = df_temp.stack().reset_index()

# 서촌의 경우 역이 늦게 생겨서 지하철 이용량이 비어있는 부분이 있음
df_s.fillna(0, inplace=True)

# 컬럼명 통일
df_s.rename(columns={0:'subway'}, inplace=True)
df_s.head()

,Date,place,subway
0,2016-01-01,강남,77201.0
1,2016-01-01,반포,25260.0
2,2016-01-01,삼청,33742.0
3,2016-01-01,성수,17678.0
4,2016-01-01,신사,62061.0


### 데이터 프레임 병합 (일별)

In [6]:
df_day = pd.merge(df_t,df_s)
df_day.head()

,Date,place,trend,subway
0,2016-01-01,홍대,47514.094171,141684.0
1,2016-01-01,연남,24403.184823,3906.0
2,2016-01-01,이태원,26406.034126,48338.0
3,2016-01-01,익선,23092.369742,85249.0
4,2016-01-01,을지로,2298.162711,101959.0


### 요일 추가
각 일자의 요일 컬럼을 추가

In [7]:
def dow(day):
    dow = ['일', '월', '화', '수', '목', '금', '토']
    return dow[day]

def to_dow(df):
    df['요일'] = pd.to_datetime(df['Date'])
    df['요일'] = df['요일'].dt.dayofweek
    df['요일'] = df['요일'].apply(dow)
   # df = df.groupby('요일').mean()
   # df = df.reindex(['월', '화', '수', '목', '금', '토', '일'])
    return df

In [8]:
df_day = to_dow(df_day)

### 비율 구하기
$ 각\,지역의\,일자별\,비율 = {각\,지역의\,일자별\,검색량\over해당\,일자의\,검색량\,총합} $

In [9]:
df_day = df_day.set_index('Date')
day_sum = df_day.groupby('Date').sum()
df_day['trend_rate'] = df_day['trend'].div(day_sum['trend'])
df_day['subway_rate'] = df_day['subway'].div(day_sum['subway'])

In [10]:
# 컬럼 순서 정렬
df_day = df_day[['요일', 'place', 'trend', 'trend_rate', 'subway', 'subway_rate']]
df_day.head()

,요일,place,trend,trend_rate,subway,subway_rate
Date,,,,,,
2016-01-01,목,홍대,47514.094171,0.173048,141684.0,0.173670
2016-01-01,목,연남,24403.184823,0.088877,3906.0,0.004788
2016-01-01,목,이태원,26406.034126,0.096172,48338.0,0.059250
2016-01-01,목,익선,23092.369742,0.084103,85249.0,0.104494
2016-01-01,목,을지로,2298.162711,0.008370,101959.0,0.124977


### 저장

In [11]:
df_day.to_csv('data/day.csv', encoding='utf-8-sig')

## 월별
부동산 매매가 데이터가 2016년 10월부터 존재하므로 이 기간동안 데이터 병합  
- 데이터 : 네이버 트렌드 검색량, 지하철 이용량, 부동산 매매가  
- 기간 : (2016-10 ~ 2021-08)

### 데이터프레임 reshape
월별로 일수가 다르므로 월별 검색량, 지하철 이용량 평균으로 group.  

In [12]:
df_temp = pd.read_csv('data/day.csv')

# YY-MM-DD 에서 -DD 부분 잘라냄
df_temp['Date'] = df_temp['Date'].apply(lambda x:x[:-3])

# 년/월 기준으로 group
df_temp = df_temp.groupby(['Date', 'place']).mean().reset_index()
df_temp.head()

,Date,place,trend,trend_rate,subway,subway_rate
0,2016-01,강남,42297.432124,0.159313,211994.096774,0.126797
1,2016-01,반포,8638.382561,0.032437,65348.516129,0.039410
2,2016-01,삼청,9205.552865,0.034392,50937.580645,0.031500
3,2016-01,성수,3208.596440,0.012084,59639.129032,0.034760
4,2016-01,신사,30090.042061,0.113136,167469.967742,0.101222


부동산 데이터 불러와서, 위 형태에 맞게 reshape

In [13]:
df_land = pd.read_csv('data/부동산/매매평균가.csv')

In [14]:
# 부동산 데이터프레임 reshape
df_land = df_land.set_index('기준년월').stack().reset_index()

# 컬럼명 통일
df_land.rename(columns={'기준년월':'Date','level_1':'place', 0:'land'}, inplace=True)
df_land.head()

,Date,place,land
0,2016-10,홍대,91727
1,2016-10,연남,52040
2,2016-10,이태원,85095
3,2016-10,익선,25014
4,2016-10,을지로,21140


### 병합

In [15]:
df_month = pd.merge(df_temp, df_land)
df_month.head()

,Date,place,trend,trend_rate,subway,subway_rate,land
0,2016-10,강남,11921.382278,0.131784,188898.741935,0.107790,92676
1,2016-10,반포,3235.554073,0.035884,91517.290323,0.051513,98194
2,2016-10,삼청,5635.684855,0.061360,59881.612903,0.034617,92852
3,2016-10,서촌,2633.491421,0.029219,58978.000000,0.032261,74917
4,2016-10,성수,2882.069656,0.031422,67410.064516,0.037718,72493


### 분기 추가

In [16]:
def to_quater(df):
    year = df['Date'].apply(lambda x:x[2:-2])
    month = df['Date'].apply(lambda x:x[-2:]).astype(int)
    quater = (1 + (month-1)//3).astype(str)
    df['quater'] = year + quater
    return df

In [17]:
df_month = to_quater(df_month)
df_month.head()

,Date,place,trend,trend_rate,subway,subway_rate,land,quater
0,2016-10,강남,11921.382278,0.131784,188898.741935,0.107790,92676,16-4
1,2016-10,반포,3235.554073,0.035884,91517.290323,0.051513,98194,16-4
2,2016-10,삼청,5635.684855,0.061360,59881.612903,0.034617,92852,16-4
3,2016-10,서촌,2633.491421,0.029219,58978.000000,0.032261,74917,16-4
4,2016-10,성수,2882.069656,0.031422,67410.064516,0.037718,72493,16-4


### 계절 추가
- 봄 : 3월, 4월, 5월
- 여름 : 6월, 7월, 8월
- 가을 : 9월, 10월, 11월
- 겨울 : 12월, 1월, 2월

In [18]:
def season(month):
    ss = ['겨울', '봄', '여름', '가을']
    if month == 12:
        return ss[0]
    return ss[month//3]

def to_season(df):
    month = df['Date'].apply(lambda x:x[-2:]).astype(int)
    df['season'] = month.apply(season)
    return df
    

In [19]:
df_month = to_season(df_month)

### 비율 구하기

In [20]:
df_month = df_month.set_index('Date')
month_sum = df_month.groupby('Date').sum()
df_month['trend_rate'] = df_month['trend'].div(month_sum['trend'])
df_month['subway_rate'] = df_month['subway'].div(month_sum['subway'])
df_month['land_rate'] = df_month['land'].div(month_sum['land'])

In [21]:
df_month = df_month[['quater', 'season', 'place', 'trend', 'trend_rate',
                     'subway', 'subway_rate', 'land', 'land_rate']]
df_month.head()

,quater,season,place,trend,trend_rate,subway,subway_rate,land,land_rate
Date,,,,,,,,,
2016-10,16-4,가을,강남,11921.382278,0.128435,188898.741935,0.106485,92676,0.079442
2016-10,16-4,가을,반포,3235.554073,0.034858,91517.290323,0.051590,98194,0.084172
2016-10,16-4,가을,삼청,5635.684855,0.060716,59881.612903,0.033756,92852,0.079592
2016-10,16-4,가을,서촌,2633.491421,0.028372,58978.000000,0.033247,74917,0.064219
2016-10,16-4,가을,성수,2882.069656,0.031050,67410.064516,0.038000,72493,0.062141


### 저장

In [22]:
df_month.to_csv('data/month.csv', encoding='utf-8-sig')